In [233]:
import pandas as pd
import plotly.express as px
import plotly.offline as py


In [4]:
df = pd.read_csv("../data_raw/lrf_public_file_review.tab", sep="\t", engine="python")

In [224]:
relevant_keys = {
    "L3_A": "In your own words, what is the greatest source of RISK TO YOUR SAFETY in your daily life?",
    "L3_B": "Other than what you just mentioned in your own words what is another MAJOR source of RISK TO YOUR SAFETY in your daily life?",
    "L5": "Do you think that climate change is a very serious threat, a somewhat serious threat, or not a threat at all to the people in this country in the next 20 years?"
}

In [6]:
survey_columns = df.columns[df.columns.str.startswith("L")].to_list()
drop_columns = [item for item in survey_columns if item not in list(relevant_keys.keys())]
df_relevant = df.drop(drop_columns, axis=1)

In [210]:
bins = [15, 25, 35, 50, 70, 100]
labels = ["15 to 24", "25 to 34", "35 to 49", "50 to 69", "70+"]

df_relevant["AgeGroup"] = pd.cut(df_relevant["Age"], bins=bins, labels=labels, right=False)

In [211]:
l3_a = df_relevant.pivot_table(index=["Country", "AgeGroup"], columns="L3_A", values="projection_weight", aggfunc='sum').astype(int)
l3_a = l3_a.rename_axis(None, axis=1)
l3_a.columns = l3_a.columns.astype(str)

l3_b = df_relevant.pivot_table(index=["Country", "AgeGroup"], columns="L3_B", values="projection_weight", aggfunc='sum').astype(int)
l3_b = l3_b.rename({" ": "99"}, axis=1)
l3_b = l3_b.rename_axis(None, axis=1)

l3 = l3_a + l3_a

l5 = df_relevant.pivot_table(index=["Country", "AgeGroup"], columns="L5", values="projection_weight", aggfunc='sum').astype(int)

In [212]:
l5_bar = l5.copy()
l5_bar["total"] = l5.sum(axis=1)
country_population = l5_bar.reset_index().groupby("Country").agg(Country=("total", "sum"))
l5_bar = l5_bar.drop([99], axis=1)
l5_bar = l5_bar.join(country_population, on="Country")

l5_bar = l5_bar[[1, 2, 3, 98]].apply(lambda x: (x/l5_bar["Country"])*100_000)
l5_bar = l5_bar.round(-1).astype(int)

In [237]:
import plotly.graph_objects as go

countries = ["United Kingdom", "Ireland", "France", "Germany", "Spain", "Netherlands", "Norway", "Sweden"]

fig = go.Figure()
buttons = []
i = 0

colors = ["rgb(253,231,37)", "rgb(53,183,121)", "rgb(49,104,142)", "rgb(68,1,84)"][::-1]

for country in countries:
    current = l5_bar.loc[country]
    current = current.rename({1: "Very serious threat", 2: "Somewhat serious threat", 3: "Not a threat at all", 98: "Don't know"}, axis=1)

    for rgb, col in enumerate(current.columns.to_list()[::-1]):

        fig.add_trace(
            go.Bar(
                x=current.index.to_list(),
                y=current[col].to_list(),
                name=col,
                visible=(i == 0),
                marker=dict(color=colors[rgb])
            )
        )

    args = [False] * len(countries) * len(current.columns)
    args[i*len(current.columns):(i*len(current.columns))+len(current.columns)] = [True] * len(current.columns)
    
    button = dict(label = country,
                  method = "update",
                  args=[{"visible": args}])
    
    buttons.append(button)
    
    i += 1

fig.update_layout(
    updatemenus=[
        dict(
            type="dropdown",
            direction="down",
            x=1,
            y=1,
            buttons=buttons
        )
    ],
    barmode="stack"
)

# Set title
fig.update_layout(title_text="Do you think that climate change is a very serious threat, a somewhat serious threat,<br>or not a threat at all to the people in this country in the next 20 years?")

fig.update_xaxes(
    title_text = "Age"
)

fig.update_yaxes(
    title_text = "People per 100,000 adults"
)

py.plot(fig, filename="../plots/concern_bar.html")

'../plots/concern_bar.html'

In [43]:
import json

with open("../data_raw/countries.geojson", "r") as f:
    country_borders = json.load(f)

In [217]:
l5_total = l5.copy()
l5_total = l5_total.groupby("Country").sum()
l5_total["% threat"] = (l5_total[1] + l5_total[2])/l5_total.sum(axis=1)

In [218]:
import pycountry

input_countries = l5_total.index.to_list()
countries = {country.name : country.alpha_3 for country in pycountry.countries}

In [219]:
countries["Vietnam"] = countries.get("Viet Nam")
countries["Russia"] = countries.get("Russian Federation")
countries["Bolivia"] = countries.get("Bolivia, Plurinational State of")
countries["Bosnia Herzegovina"] = countries.get("Bosnia and Herzegovina")
countries["Congo Brazzaville"] = countries.get("Congo, The Democratic Republic of the")
countries["Iran"] = countries.get("Iran, Islamic Republic of")
countries["Ivory Coast"] = countries.get("Iran, Islamic Republic of")
countries["Iran"] = countries.get("Côte d\'Ivoire")
countries["Laos"] = countries["Lao People\'s Democratic Republic"]
countries["Palestine"] = countries["Palestine, State of"]
countries["South Korea"] = countries["Korea, Republic of"]
countries["Taiwan"] = countries["Taiwan, Province of China"]
countries["Taiwan"] = countries["Taiwan, Province of China"]
countries["Tanzania"] = countries["Tanzania, United Republic of"]
countries["Venezuela"] = countries["Venezuela, Bolivarian Republic of"]

l5_total["codes"] = [countries.get(country) for country in input_countries]
l5_map = l5_total.reset_index()

In [220]:
l5_map["% threat"] = l5_map["% threat"]*100
l5_map["% threat"] = l5_map["% threat"].round(0)

In [235]:
px.set_mapbox_access_token("pk.eyJ1IjoibHVpc2RpYmRpbiIsImEiOiJjbGE5bDgyM28wZXdrM3dsdmZqdGVucXN3In0.ZwFwkorjA9Pa7q1dKdHuIA")

fig = px.choropleth_mapbox(l5_map, geojson=country_borders, locations="codes", featureidkey="properties.ISO_A3", color="% threat", color_continuous_scale="Viridis", hover_name="Country", hover_data={"codes":False, "% threat": True})

fig.update_layout(
    title_text="Percent of people who believe climate change will be very serious or somewhat serious threat in the next 20 years.",
    autosize=False,
    margin=dict(l=0, r=0, t=0, b=0),
    height=800,
    width=1000,
    mapbox_zoom=1.5,
    mapbox_center = {"lat": 35, "lon": 10})

py.plot(fig, filename="../plots/concern_map.html")


'../plots/concern_map.html'